# Settle Up Problem (Dlužníček)

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [7]:
P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values())/len(c)  # the settlement value

Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x_k, y_k, z_k)$ representing the transactions: person $x_k$ should pay person $y_k$ $z_k$ euros. The number of transactions (i.e., the length of the list) should be minimized.

For the given instance, the **optimal solution** consists of 3 transactions, namely

```
A -> B: 250.0
C -> D: 50.0
C -> B: 90.0
```

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [8]:
import gurobipy as g  # import Gurobi module


# model --------------------------------------------------
m = g.Model()

# - ADD VARIABLES
R = m.addVars(P, P, vtype=g.GRB.CONTINUOUS, lb=0, name="R")
x = m.addVars(P, P, vtype=g.GRB.BINARY, name="x")

# - ADD CONSTRAINTS
m.addConstrs(sv == c[i] + R.sum(i, "*") - R.sum("*", i)
             for i in P)
M = sum(c.values())
m.addConstrs(R[i,j] <= M*x[i,j]
             for i in P
             for j in P)

# - SET OBJECTIVE
m.setObjective(x.sum("*", "*"))

# call the solver -------------------------------------------
m.optimize()

# print the solution -----------------------------------------
print('\nSOLUTION:')
for i in P:
    for j in P:
        print(R[i,j].X)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 32 columns and 56 nonzeros
Model fingerprint: 0xc9365600
Variable types: 16 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 4 rows and 8 columns
Presolve time: 0.00s
Presolved: 16 rows, 24 columns, 48 nonzeros
Variable types: 12 continuous, 12 integer (12 binary)

Root relaxation: objective 3.900000e-01, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.39000    0    3    3.00000    0.39000  87.0%     -    0s
     0     0     cutoff    0         3.00000   

## Additional experiments

* experiment with different values of the 'big M' constant
* try to generalize the model to work even with float sv (e.g., rounded to 0.01)
 